In [1]:
import pandas as pd
import string
import nltk
import numpy as np
import keras
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.models import Sequential
import pickle
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation, TimeDistributed, Bidirectional
from keras.layers.embeddings import Embedding
from keras import optimizers
from keras.layers import Input
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


# Start here

In [67]:
#Save checkpoint
# data.to_csv(r"E:\Yelp\Unfiltered Data\YelpZip\Customs\checkpoint", header=True, index=False)

#Use if kernel crashes
data = pd.read_csv(r"E:\Yelp\Unfiltered Data\YelpZip\Customs\checkpoint")
data.head()

,index,user,item,review,rating,real,date
0,0,5044,0,drink be bad the hot chocolate be water down a...,1.0,-1,2014-11-16
1,1,5045,0,this be the bad experience ive ever have a cas...,1.0,-1,2014-09-08
2,2,5046,0,this be locate on the site of the old spruce s...,3.0,-1,2013-10-06
3,3,5047,0,i enjoy coffee and breakfast twice at toast du...,5.0,-1,2014-11-30
4,4,5048,0,i love toast the food choice be fantastic i lo...,5.0,-1,2014-08-28


In [68]:
data['review'] = data['review'].apply(lambda s: str(s))

In [69]:
#1 for good
#0 for bad
def binarize(n):
    if n==-1:
        return 1
    else:
        return 0

data['real'] = data['real'].apply(lambda s: binarize(s))

In [70]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data['review'], data['real'], test_size=0.30, random_state=42)

In [71]:
data.groupby('real').count()

,index,user,item,review,rating,date
real,,,,,,
0,528019,528019,528019,528019,528019,528019
1,80439,80439,80439,80439,80439,80439


In [72]:
tokenizer = Tokenizer(num_words= 2000)
tokenizer.fit_on_texts(data['review'])
sequences = tokenizer.texts_to_sequences(X_train)
data1 = pad_sequences(sequences, maxlen=100)

In [73]:
# inputs = Input(shape=[100])
# layers = Embedding(2000,32,input_length=100)(inputs)
# layers = LSTM(128, recurrent_dropout=0.2)(layers)
# layers = Dense(1)(layers)
# layers = Activation('sigmoid')(layers)
# model = Model(inputs=inputs,outputs=layers)
# model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

# def create_conv_model():
#     model_conv = Sequential()
#     model_conv.add(Embedding(vocabulary_size, 100, input_length=50))
#     model_conv.add(Dropout(0.2))
#     model_conv.add(Conv1D(64, 5, activation='relu'))
#     model_conv.add(MaxPooling1D(pool_size=4))
#     model_conv.add(LSTM(100))
#     model_conv.add(Dense(1, activation='sigmoid'))
#     model_conv.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#     return model_conv


model = Sequential()
model.add(Embedding(2000, 32, input_length=100))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 100, 32)           64000     
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 100, 32)           3104      
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 50, 32)            0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 50, 32)            3104      
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 25, 32)            0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 800)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 256)               205056    
__________

In [74]:
model.fit(data1,y_train,batch_size=1000,epochs=1, verbose = 1)

Epoch 1/1


194000/425920 [============>.................] - ETA: 3:21:48 - loss: 0.7012 - acc: 0.13 - ETA: 3:11:34 - loss: 0.6558 - acc: 0.50 - ETA: 3:09:00 - loss: 0.5862 - acc: 0.62 - ETA: 3:07:28 - loss: 0.5248 - acc: 0.69 - ETA: 3:02:22 - loss: 0.4968 - acc: 0.72 - ETA: 3:01:36 - loss: 0.4768 - acc: 0.75 - ETA: 3:00:20 - loss: 0.4637 - acc: 0.76 - ETA: 2:59:14 - loss: 0.4515 - acc: 0.78 - ETA: 2:57:27 - loss: 0.4453 - acc: 0.78 - ETA: 2:55:46 - loss: 0.4341 - acc: 0.79 - ETA: 2:56:52 - loss: 0.4333 - acc: 0.80 - ETA: 2:55:42 - loss: 0.4301 - acc: 0.81 - ETA: 2:55:47 - loss: 0.4291 - acc: 0.81 - ETA: 2:55:38 - loss: 0.4265 - acc: 0.81 - ETA: 2:55:04 - loss: 0.4246 - acc: 0.82 - ETA: 2:55:50 - loss: 0.4213 - acc: 0.82 - ETA: 2:56:45 - loss: 0.4190 - acc: 0.82 - ETA: 2:55:35 - loss: 0.4167 - acc: 0.82 - ETA: 2:55:01 - loss: 0.4155 - acc: 0.83 - ETA: 2:54:25 - loss: 0.4159 - acc: 0.83 - ETA: 2:53:48 - loss: 0.4145 - acc: 0.83 - ETA: 2:53:58 - loss: 0.4140 - acc: 0.83 - ETA: 2:52:54 - loss: 0.4119

393000/425920 [==========================>...] - ETA: 1:40:06 - loss: 0.3705 - acc: 0.86 - ETA: 1:39:42 - loss: 0.3704 - acc: 0.86 - ETA: 1:39:14 - loss: 0.3704 - acc: 0.86 - ETA: 1:38:50 - loss: 0.3704 - acc: 0.86 - ETA: 1:38:23 - loss: 0.3703 - acc: 0.86 - ETA: 1:37:55 - loss: 0.3702 - acc: 0.86 - ETA: 1:37:30 - loss: 0.3701 - acc: 0.86 - ETA: 1:37:06 - loss: 0.3700 - acc: 0.86 - ETA: 1:36:39 - loss: 0.3700 - acc: 0.86 - ETA: 1:36:14 - loss: 0.3699 - acc: 0.86 - ETA: 1:35:47 - loss: 0.3699 - acc: 0.86 - ETA: 1:35:20 - loss: 0.3701 - acc: 0.86 - ETA: 1:34:56 - loss: 0.3703 - acc: 0.86 - ETA: 1:34:30 - loss: 0.3702 - acc: 0.86 - ETA: 1:34:05 - loss: 0.3702 - acc: 0.86 - ETA: 1:33:38 - loss: 0.3701 - acc: 0.86 - ETA: 1:33:11 - loss: 0.3700 - acc: 0.86 - ETA: 1:32:45 - loss: 0.3700 - acc: 0.86 - ETA: 1:32:20 - loss: 0.3700 - acc: 0.86 - ETA: 1:31:55 - loss: 0.3700 - acc: 0.86 - ETA: 1:31:28 - loss: 0.3700 - acc: 0.86 - ETA: 1:31:02 - loss: 0.3700 - acc: 0.86 - ETA: 1:30:34 - loss: 0.3700

425920/425920 [==============================] - ETA: 13:52 - loss: 0.3639 - acc: 0.86 - ETA: 13:26 - loss: 0.3639 - acc: 0.86 - ETA: 13:00 - loss: 0.3638 - acc: 0.86 - ETA: 12:34 - loss: 0.3638 - acc: 0.86 - ETA: 12:08 - loss: 0.3638 - acc: 0.86 - ETA: 11:42 - loss: 0.3638 - acc: 0.86 - ETA: 11:16 - loss: 0.3637 - acc: 0.86 - ETA: 10:49 - loss: 0.3637 - acc: 0.86 - ETA: 10:23 - loss: 0.3637 - acc: 0.86 - ETA: 9:57 - loss: 0.3638 - acc: 0.8662 - ETA: 9:31 - loss: 0.3638 - acc: 0.866 - ETA: 9:05 - loss: 0.3638 - acc: 0.866 - ETA: 8:39 - loss: 0.3639 - acc: 0.866 - ETA: 8:13 - loss: 0.3638 - acc: 0.866 - ETA: 7:47 - loss: 0.3638 - acc: 0.866 - ETA: 7:21 - loss: 0.3638 - acc: 0.866 - ETA: 6:55 - loss: 0.3638 - acc: 0.866 - ETA: 6:29 - loss: 0.3637 - acc: 0.866 - ETA: 6:02 - loss: 0.3637 - acc: 0.866 - ETA: 5:36 - loss: 0.3637 - acc: 0.866 - ETA: 5:10 - loss: 0.3637 - acc: 0.866 - ETA: 4:44 - loss: 0.3636 - acc: 0.866 - ETA: 4:18 - loss: 0.3637 - acc: 0.866 - ETA: 3:52 - loss: 0.3637 - acc

In [75]:
test_sequences = tokenizer.texts_to_sequences(X_test)
test_sequences_matrix = pad_sequences(test_sequences,maxlen=100)

In [76]:
# accr = model.evaluate(test_sequences_matrix,y_test)
y_pred = model.predict(test_sequences_matrix)

In [96]:
tst = []
for i in y_pred:
    if i >= 0.20:
        tst.append(1)
    else:
        tst.append(0)

from sklearn.metrics import classification_report
print(classification_report(y_test, tst))

             precision    recall  f1-score   support

          0       0.89      0.91      0.90    158422
          1       0.34      0.30      0.31     24116

avg / total       0.82      0.83      0.83    182538



In [97]:
results = model.evaluate(test_sequences_matrix, y_test)

182538/182538 [==============================] - ETA: 51 - ETA: 34 - ETA: 28 - ETA: 26 - ETA: 24 - ETA: 24 - ETA: 23 - ETA: 23 - ETA: 23 - ETA: 22 - ETA: 22 - ETA: 22 - ETA: 21 - ETA: 21 - ETA: 21 - ETA: 21 - ETA: 21 - ETA: 21 - ETA: 21 - ETA: 21 - ETA: 20 - ETA: 20 - ETA: 20 - ETA: 20 - ETA: 20 - ETA: 20 - ETA: 20 - ETA: 20 - ETA: 20 - ETA: 20 - ETA: 19 - ETA: 19 - ETA: 19 - ETA: 19 - ETA: 19 - ETA: 19 - ETA: 19 - ETA: 19 - ETA: 19 - ETA: 19 - ETA: 19 - ETA: 19 - ETA: 18 - ETA: 18 - ETA: 18 - ETA: 18 - ETA: 18 - ETA: 18 - ETA: 18 - ETA: 18 - ETA: 18 - ETA: 18 - ETA: 18 - ETA: 18 - ETA: 18 - ETA: 18 - ETA: 18 - ETA: 17 - ETA: 17 - ETA: 17 - ETA: 17 - ETA: 17 - ETA: 17 - ETA: 17 - ETA: 17 - ETA: 17 - ETA: 17 - ETA: 17 - ETA: 17 - ETA: 17 - ETA: 17 - ETA: 17 - ETA: 17 - ETA: 17 - ETA: 16 - ETA: 16 - ETA: 16 - ETA: 16 - ETA: 16 - ETA: 16 - ETA: 16 - ETA: 16 - ETA: 16 - ETA: 16 - ETA: 16 - ETA: 16 - ETA: 16 - ETA: 16 - ETA: 16 - ETA: 16 - ETA: 16 - ETA: 16 - ETA: 15 - ETA: 15 - ETA: 15 - E

In [84]:
print("Accuracy:", results[1]*100, "%")

Accuracy: 86.81260888150369 %


In [34]:
b/a

0.14800052828693527